In [2]:
import malaya_speech
import malaya

`openai-whisper` is not available, native whisper processor is not available, will use huggingface processor instead.
`pyaudio` is not available, `malaya_speech.streaming.pyaudio` is not able to use.


In [3]:
true_case = malaya.true_case.huggingface(model = 'mesolitica/finetune-true-case-t5-small-standard-bahasa-cased')
_ = true_case.cuda()

Downloading:   0%|          | 0.00/2.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/803k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/822 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [4]:
model = malaya_speech.force_alignment.transducer.transformer(device = 'gpu:0')

Downloading:   0%|          | 0.00/120M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.49k [00:00<?, ?B/s]

2023-03-05 15:34:50.885418: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-05 15:34:50.888142: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-05 15:34:50.908224: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-05 15:34:50.908940: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [5]:
def put(alignment, length, min_threshold = 0.5, min_threshold_fullstop = 1.0):
    r = []
    for no, row in enumerate(alignment):
        if no > 0:
            if alignment[no]['start'] - alignment[no-1]['end'] >= min_threshold:
                r.append(',')

        r.append(row['text'])
    if length - alignment[-1]['end'] >= min_threshold_fullstop:
        r.append('.')
    else:
        r.append(',')
    return r

In [9]:
from datasets import Audio

sr = 16000
audio = Audio(sampling_rate=sr)

In [8]:
import json

with open('semisupervised-asr.json') as fopen:
    data = json.load(fopen)

In [17]:
from tqdm import tqdm

new_results = {}

for k, v in tqdm(data.items()):
    try:
        v = v[0]
        y = audio.decode_example(audio.encode_example(k))['array']
        result = model.predict(y, v)
        if ' '.join([r['text'] for r in result['words_alignment']]) != v:
            continue
        o = put(result['words_alignment'], len(y) / sr)
        o = ' '.join(o)
        o_true_case = true_case.generate([o], max_length = 256)[0]

        new_results[k] = {
            'text': v,
            'force_alignment': o,
            'true_case': o_true_case,
        }
    except Exception as e:
        print(e)

 16%|███████████████▎                                                                                 | 599/3783 [03:12<12:16,  4.32it/s]

list index out of range


 25%|███████████████████████▉                                                                         | 934/3783 [04:47<10:24,  4.57it/s]

list index out of range


 44%|██████████████████████████████████████████▏                                                     | 1663/3783 [08:07<06:30,  5.43it/s]

...

 99%|██████████████████████████████████████████████████████████████████████████████████████████████▌ | 3728/3783 [17:10<00:13,  4.05it/s]

list index out of range


 99%|██████████████████████████████████████████████████████████████████████████████████████████████▉ | 3739/3783 [17:13<00:13,  3.33it/s]

list index out of range


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 3783/3783 [17:23<00:00,  3.62it/s]


In [19]:
with open('comma-true-case.json', 'w') as fopen:
    json.dump(new_results, fopen)